In [1]:
import pandas as pd
import numpy as np
import os
import json
import gspread as gs
import warnings

from clean_argy import argy_wrangling

pd.options.display.max_columns = None
pd.options.display.max_rows = None
pd.options.display.max_colwidth = None

warnings.simplefilter("ignore")

                ### ~~~ ###
            ###     PATH     ###
                ### ~~~ ###

location_2013 = "./data/283525_Importaciones_2013.xlsx"
location_2014 = "./data/283525_Importaciones_2014.xlsx"
location_2015 = "./data/283525_Importaciones_2015.xlsx"
location_2016 = "./data/283525_Importaciones_2016.xlsx"
location_2017 = "./data/283525_Importaciones_2017.xlsx"
location_2018 = "./data/283525_Importaciones_2018.xlsx"
location_2019 = "./data/283525_Importaciones_2019.xlsx"
location_2020 = "./data/283525_Importaciones_2020.xlsx"
location_2021 = "./data/283525_Importaciones_2021.xlsx"
location_2022 = "./data/283525_Importaciones_2022.xlsx"
location_2023 = "./data/283525_Importaciones_2023.xlsx"

                ### ~~~ ###
            ###     DATA     ###
                ### ~~~ ###

data2013 = pd.read_excel(location_2013, engine="openpyxl")
data2014 = pd.read_excel(location_2014, engine="openpyxl")
data2015 = pd.read_excel(location_2015, engine="openpyxl")
data2016 = pd.read_excel(location_2016, engine="openpyxl")
data2017 = pd.read_excel(location_2017, engine="openpyxl")
data2018 = pd.read_excel(location_2018, engine="openpyxl")
data2019 = pd.read_excel(location_2019, engine="openpyxl")
data2020 = pd.read_excel(location_2020, engine="openpyxl")
data2021 = pd.read_excel(location_2021, engine="openpyxl")
data2022 = pd.read_excel(location_2022, engine="openpyxl")
data2023 = pd.read_excel(location_2023, engine="openpyxl")



In [2]:
# MERGE
dfs = [data2013, data2014, data2015, data2016, data2017, data2018, data2019, data2020, data2021, data2022, data2023]

print("Este df tiene (filas, col):\n___________________")

for df in dfs:
    print(f"> {df['Fecha'][0].year}: {df.shape}\n~~~~~~~~~~~~~~~~~~~")
    


Este df tiene (filas, col):
___________________
> 2013: (84, 35)
~~~~~~~~~~~~~~~~~~~
> 2014: (99, 35)
~~~~~~~~~~~~~~~~~~~
> 2015: (79, 35)
~~~~~~~~~~~~~~~~~~~
> 2016: (75, 35)
~~~~~~~~~~~~~~~~~~~
> 2017: (66, 35)
~~~~~~~~~~~~~~~~~~~
> 2018: (70, 35)
~~~~~~~~~~~~~~~~~~~
> 2019: (70, 35)
~~~~~~~~~~~~~~~~~~~
> 2020: (63, 35)
~~~~~~~~~~~~~~~~~~~
> 2021: (67, 35)
~~~~~~~~~~~~~~~~~~~
> 2022: (71, 35)
~~~~~~~~~~~~~~~~~~~
> 2023: (9, 35)
~~~~~~~~~~~~~~~~~~~


In [3]:
argy_wrangling(dfs)

# df_final = pd.concat(dfs, axis=0)
# df_final.isnull().sum()
# df_final.sample(25)


> Done with: 2013
~~~~~~~~~~~~~~~~~~~
> Done with: 2014
~~~~~~~~~~~~~~~~~~~
> Done with: 2015
~~~~~~~~~~~~~~~~~~~
> Done with: 2016
~~~~~~~~~~~~~~~~~~~
> Done with: 2017
~~~~~~~~~~~~~~~~~~~
> Done with: 2018
~~~~~~~~~~~~~~~~~~~
> Done with: 2019
~~~~~~~~~~~~~~~~~~~
> Done with: 2020
~~~~~~~~~~~~~~~~~~~
> Done with: 2021
~~~~~~~~~~~~~~~~~~~
> Done with: 2022
~~~~~~~~~~~~~~~~~~~
> Done with: 2023
~~~~~~~~~~~~~~~~~~~


In [4]:
def last_year_data(df):

    transition_dic = {
        "NCM": [],
        "Año": [],
        "Importadores": [],
        "Origenes": [],
    }

    for df in dfs:
        transition_dic['NCM'].append(''.join(df['NCM-SIM'].unique()))

        transition_dic['Año'].append(df["Fecha"][0].year)

        volumenTotalImportacionTn = (df['Kgs. Netos'].sum()/1000).round(2)

        transition_dic['Volumen Total (Tn)'].append(volumenTotalImportacionTn)

        for company in df['Importador'].unique():
            global company_dic

            company_dic = {
                "Importador": [],
                "PrecioPromedio": [],
                "VolumenTotal": [],
                "Participacion": []
            }

            data = df[df['Importador'] == f"{company}"]

            if (data['Kgs. Netos'].sum() > 15000):

                volumenTotalCompania = (data['Kgs. Netos'].sum()/1000).round(2)

                # Adding iterated values to the dataframe

                company_dic['Importador'].append(company)

                company_dic['PrecioPromedio'].append((data['U$S Unitario'].mean().round(2))*1000)
                
                company_dic['VolumenTotal'].append(volumenTotalCompania)

                company_dic['Participacion en Vol. Total'].append(f"{round((volumenTotalCompania / volumenTotalImportacionTn) * 100)}%")

    transition_df = pd.DataFrame.from_records(transition_dic)
    transition_df = transition_df.sort_values(
        by="Volumen Total (Tn)", ascending=False)
    excel_new_data = excel_new_data.append(transition_df)


In [5]:
# EXPORT TO PICKLE
CURRENT_WD = os.getcwd()
COUNTRY_WD = os.path.basename(CURRENT_WD).split('/')[-1]

# df_final.to_pickle(f'{COUNTRY_WD}.pkl')
